## Protodash Explanations for Text data

In the example shown in this notebook, we train a text classifier based on [UCI SMS dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) to distinguish 'SPAM' and 'HAM' (i.e. non spam) SMS messages. We then use the ProtodashExplainer to obtain spam and ham prototypes based on the labels assigned by the text classifier. 

In order to run this notebook, please: 
1. Download [UCI SMS dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) dataset and place the directory 'smsspamcollection' in the location of this notebook. 
2. Place glove embedding file "glove.6B.100d.txt" in the location of this notebook. This can be downloaded from [here](https://nlp.stanford.edu/projects/glove/) 
3. Create 2 folders: "results" and "logs" in the location of this notebook (these are used to store training logs). 
4. The models trained in this notebook can also be accessed from [here](https://github.com/IBM/AIX360/tree/master/aix360/models/protodash) if required. 

### Step 1. Train a LSTM classifier on SMS dataset
We train a LSTM model to label the dataset as spam / ham. The model is based on the following code: https://www.thepythoncode.com/article/build-spam-classifier-keras-python 

#### Import statements

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tqdm
import numpy as np
import keras_metrics # for recall and precision metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dropout, Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pickle
import os.path
from keras.models import model_from_json

In [2]:
SEQUENCE_LENGTH = 100 # the length of all sequences (number of words per sample)
EMBEDDING_SIZE = 100  # Using 100-Dimensional GloVe embedding vectors
TEST_SIZE = 0.25 # ratio of testing set

BATCH_SIZE = 64
EPOCHS = 20 # number of epochs

# to convert labels to integers and vice-versa
label2int = {"ham": 0, "spam": 1}
int2label = {0: "ham", 1: "spam"}

In [3]:
import pandas as pd
combined_df = pd.read_csv("C:/Users/hp/Documents/ML/jupyter notebook/Datasets/SMSSpamCollection.txt", delimiter='\t',header=None)
combined_df.columns = ['label', 'text']
combined_df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
# Divide in text and labels list from original df
X = combined_df['text'].values.tolist()
y = combined_df['label'].values.tolist()

In [5]:
# Text tokenization, extract features from whole X list  
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)                         

# convert to sequence of integers
X = tokenizer.texts_to_sequences(X)

In [6]:
# pad sequences at the beginning of each sequence with 0's
# for example if SEQUENCE_LENGTH=4:
# [[5, 3, 2], [5, 1, 2, 3], [3, 4]]
# will be transformed to:
# [[0, 5, 3, 2], [5, 1, 2, 3], [0, 0, 3, 4]]
X = pad_sequences(X, maxlen=SEQUENCE_LENGTH)

# convert to numpy arrays
X = np.array(X)
y = np.array(y)

In [7]:
y = [ label2int[label] for label in y ]
y = to_categorical(y)

In [8]:
y

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [9]:
# split and shuffle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=7)

#### Use glove embeddings

In [10]:
def get_embedding_vectors(tokenizer, dim=100):
    embedding_index = {}
    with open(f"glove.6B.{dim}d.txt", encoding='utf8') as f:
        for line in tqdm.tqdm(f, "Reading GloVe"):
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = vectors

    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index)+1, dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # words not found will be 0s
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In [11]:
def get_model(tokenizer, lstm_units):
    """
    Constructs the model,
    Embedding vectors => LSTM => 2 output Fully-Connected neurons with softmax activation
    """
    # get the GloVe embedding vectors
    embedding_matrix = get_embedding_vectors(tokenizer)
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index)+1,
              EMBEDDING_SIZE,
              weights=[embedding_matrix],
              trainable=False,
              input_length=SEQUENCE_LENGTH))

    model.add(LSTM(lstm_units, recurrent_dropout=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation="softmax"))
    # compile as rmsprop optimizer
    # aswell as with recall metric
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",
                  metrics=["accuracy", keras_metrics.precision(), keras_metrics.recall()])
    model.summary()
    return model

In [12]:
# constructs the model with 128 LSTM units
model = get_model(tokenizer=tokenizer, lstm_units=128)

Reading GloVe: 400000it [00:22, 17569.44it/s]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          901200    
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 1018706 (3.89 MB)
Trainable params: 117506 (459.01 KB)
Non-trainable params: 901200 (3.44 MB)
_________________________________________________________________


#### Train model or load trained model from disk

In [13]:
to_train = False

if (to_train): 

    # initialize our ModelCheckpoint and TensorBoard callbacks
    # model checkpoint for saving best weights
    
    model_checkpoint = ModelCheckpoint("results/spam_classifier_{val_loss:.2f}", save_best_only=True,verbose=1)
    # for better visualization
    tensorboard = TensorBoard(f"logs/spam_classifier_{time.time()}")
    # print our data shapes
    print("X_train.shape:", X_train.shape)
    print("X_test.shape:", X_test.shape)
    print("y_train.shape:", y_train.shape)
    print("y_test.shape:", y_test.shape)
    # train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=BATCH_SIZE, epochs=EPOCHS,callbacks=[tensorboard, model_checkpoint],verbose=1)
      # serialize model to JSON
    model_json = model.to_json()
    with open("sms-lstm-forprotodash.json", "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights("sms-lstm-forprotodash.h5")
    print("Saved model to disk")
        
else: 

    # load json and create model
    json_file = open("sms-lstm-forprotodash.json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)

    # load weights into new model
    model.load_weights("sms-lstm-forprotodash.h5")
    print("Loaded model from disk")  
    # print model 
    model.summary()

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=["accuracy"])

Loaded model from disk
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          901000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1018506 (3.89 MB)
Trainable params: 117506 (459.01 KB)
Non-trainable params: 901000 (3.44 MB)
_________________________________________________________________


In [14]:
# get the loss and metrics
result = model.evaluate(X_test, y_test)

# extract those
loss = result[0]
accuracy = result[1]

print(f"[+] Accuracy: {accuracy*100:.2f}%")

44/44 [==============================] - 4s 61ms/step - loss: 0.0766 - accuracy: 0.9813
[+] Accuracy: 98.13%


### Step 2. Get model predictions for the dataset

In [15]:
def get_predictions(doclist):
    
    sequence = tokenizer.texts_to_sequences(doclist)
    
    # pad the sequence
    sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
    # get the prediction as one-hot encoded vector
    prediction = model.predict(sequence)
    
    return (prediction)    

In [16]:
text1 = "Congratulations! you have won 100,000$ this week, click here to claim fast"
pred = get_predictions([text1])
print(int2label [ np.argmax(pred, axis=1)[0] ] )

1/1 [==============================] - 1s 976ms/step
spam


In [17]:
text2 = "Hi man, I was wondering if we can meet tomorrow."
pred = get_predictions([text2,text1])
print(int2label [ np.argmax(pred, axis=1)[0] ] )
print(int2label [ np.argmax(pred, axis=1)[1] ] )

1/1 [==============================] - 0s 72ms/step
ham
spam


In [18]:
doclist = combined_df['text'].values.tolist()
one_hot_prediction = get_predictions(doclist[:-10])
label_prediction = np.argmax(one_hot_prediction, axis=1)

# 0: ham, 1:spam
idx_ham = (label_prediction == 0)
idx_spam = (label_prediction == 1)

174/174 [==============================] - 10s 58ms/step


###  Step 3. Use protodash explainer to compute spam and ham prototypes

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from aix360.algorithms.protodash import ProtodashExplainer

#### Convert text to vectors using TF-IDF for use in explainer

We use TF-IDF vectors for scalability reasons as the original embedding vector for a full sentence can be quite large. 

In [20]:
# create the transform
vectorizer = TfidfVectorizer()

# tokenize and build vocab
vectorizer.fit(doclist[:-10])

vec = vectorizer.transform(doclist[:-10])
docvec = vec.toarray()
print(docvec.shape)

(5562, 8707)


In [21]:
# separate spam and ham messages and corrsponding vectors
new_df=combined_df[:-10]
docvec_spam = docvec[idx_spam, :]
docvec_ham = docvec[idx_ham, :]
df_spam = new_df[idx_spam]['text']
df_ham = new_df[idx_ham]['text']

In [22]:
print(df_spam.shape)
print(df_ham.shape)

(717,)
(4845,)


#### Compute prototypes for spam and ham datasets

In [23]:
explainer = ProtodashExplainer()

In [24]:
m = 10

# call protodash explainer
# S contains indices of the selected prototypes
# W contains importance weights associated with the selected prototypes 
(W_spam, S_spam, _) = explainer.explain(docvec_spam, docvec_spam, m=m)
(W_ham, S_ham, _) = explainer.explain(docvec_ham, docvec_ham, m=m)

In [25]:
# get prototypes from index
df_spam_prototypes = df_spam.iloc[S_spam].copy()
df_ham_prototypes = df_ham.iloc[S_ham].copy()

#normalize weights
W_spam = np.around(W_spam/np.sum(W_spam), 2) 
W_ham = np.around(W_ham/np.sum(W_ham), 2) 

In [26]:
print("SPAM prototypes with weights:")
print("----------------------------")
for i in range(m):
    print(W_spam[i], df_spam_prototypes.iloc[i])

SPAM prototypes with weights:
----------------------------
0.13 We tried to call you re your reply to our sms for a video mobile 750 mins UNLIMITED TEXT + free camcorder Reply of call 08000930705 Now
0.13 You have WON a guaranteed £1000 cash or a £2000 prize.To claim yr prize call our customer service representative on
0.12 Get ur 1st RINGTONE FREE NOW! Reply to this msg with TONE. Gr8 TOP 20 tones to your phone every week just £1.50 per wk 2 opt out send STOP 08452810071 16
0.1 December only! Had your mobile 11mths+? You are entitled to update to the latest colour camera mobile for Free! Call The Mobile Update Co FREE on 08002986906
0.09 Dear Voucher Holder, To claim this weeks offer, at you PC please go to http://www.e-tlp.co.uk/expressoffer Ts&Cs apply. To stop texts, txt STOP to 80062
0.09 URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only
0.08 Free entry in 2 a weekly comp fo

In [27]:
print("HAM prototypes with weights:")
print("----------------------------")
for i in range(m):
    print(W_ham[i], df_ham_prototypes.iloc[i])

HAM prototypes with weights:
----------------------------
0.14 The last thing i ever wanted to do was hurt you. And i didn't think it would have. You'd laugh, be embarassed, delete the tag and keep going. But as far as i knew, it wasn't even up. The fact that you even felt like i would do it to hurt you shows you really don't know me at all. It was messy wednesday, but it wasn't bad. The problem i have with it is you HAVE the time to clean it, but you choose not to. You skype, you take pictures, you sleep, you want to go out. I don't mind a few things here and there, but when you don't make the bed, when you throw laundry on top of it, when i can't have a friend in the house because i'm embarassed that there's underwear and bras strewn on the bed, pillows on the floor, that's something else. You used to be good about at least making the bed.
0.11 What do u want when i come back?.a beautiful necklace as a token of my heart for you.thats what i will give but ONLY to MY WIFE OF MY LIKING.

#### Given a message, look for similar messages that are classified as spam by classifier

In [28]:
k = 0
sample_text = df_spam.iloc[k]
sample_vec = docvec_spam[k]
sample_vec = sample_vec.reshape(1, sample_vec.shape[0])

In [29]:
print(sample_text)
print(sample_vec.shape)

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
(1, 8707)


In [30]:
docvec_spam_other = docvec_spam[np.arange(docvec_spam.shape[0]) != k, :] 
df_spam_other = df_spam.iloc[np.arange(docvec_spam.shape[0]) != k].copy()

In [31]:
# Take a sample spam text and find samples similar to it. 
(W1_spam, S1_spam, _) = explainer.explain(sample_vec, docvec_spam_other, m=m) 

In [32]:
#normalize weights
W1_spam = np.around(W1_spam/np.sum(W1_spam), 2) 

In [33]:
S1_spam

array([170, 255, 399, 445, 673, 512,  35,  58,  49,  37])

In [34]:
# similar spam prototypes
print("original text")
print("-------------")
print(sample_text)
print("")

print("Similar SPAM prototypes:")
print("------------------------")
m = 10
for i in range(m):
    print(W1_spam[i], df_spam_other.iloc[S1_spam[i]])

original text
-------------
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

Similar SPAM prototypes:
------------------------
1.0 Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
0.0 Free entry in 2 a weekly comp for a chance to win an ipod. Txt POD to 80182 to get entry (std txt rate) T&C's apply 08452810073 for details 18+
-0.0 Free entry in 2 a weekly comp for a chance to win an ipod. Txt POD to 80182 to get entry (std txt rate) T&C's apply 08452810073 for details 18+
-0.0 Wan2 win a Meet+Greet with Westlife 4 U or a m8? They are currently on what tour? 1)Unbreakable, 2)Untamed, 3)Unkempt. Text 1,2 or 3 to 83049. Cost 50p +std text
-0.0 ree entry in 2 a weekly comp for a chance to win an ipod. Txt POD to 80182 to get entry (std txt rate) T&C's apply 08452810073 for detai

#### Observation

Note several spam messages repeat in the dataset as these may have been sent by the same entity to multiple users. As a consequence, the explainer retireves these. Try with a different k above to see prototypes corrsponding to other sample messages. 

#### Given a ham message, look for similar messages that are classified as spam by classifier

In [35]:
k = 3
sample_text = df_ham.iloc[k]
sample_vec = docvec_ham[k]
sample_vec = sample_vec.reshape(1, sample_vec.shape[0])

In [36]:
print(sample_text)
print(sample_vec.shape)

Nah I don't think he goes to usf, he lives around here though
(1, 8707)


In [37]:
docvec_ham_other = docvec_ham[np.arange(docvec_ham.shape[0]) != k, :] 
df_ham_other = df_ham.iloc[np.arange(docvec_ham.shape[0]) != k].copy()

In [38]:
# Take a sample spam text and find samples similar to it. 
(W1_ham, S1_ham, _) = explainer.explain(sample_vec, docvec_ham_other, m=m) 

In [39]:
#normalize weights
W1_ham = np.around(W1_ham/np.sum(W1_ham), 2) 

In [40]:
S1_ham

array([ 925, 3271,  832,  345, 2352, 3826, 3586, 2137, 2465,  946])

In [41]:
# similar spam prototypes
print("original text")
print("-------------")
print(sample_text)
print("")

print("Similar HAM prototypes:")
print("------------------------")
m = 10
for i in range(m):
    print(W1_ham[i], df_ham_other.iloc[S1_ham[i]])

original text
-------------
Nah I don't think he goes to usf, he lives around here though

Similar HAM prototypes:
------------------------
0.13 I don't think he has spatula hands!
0.13 Aight, let me know when you're gonna be around usf
0.1 If he started searching he will get job in few days.he have great potential and talent.
0.09 None of that's happening til you get here though
0.11 Nah, I'm a perpetual DD
0.1 Yes just finished watching days of our lives. I love it.
0.1 Babe! How goes that day ? What are you up to ? I miss you already, my Love ... * loving kiss* ... I hope everything goes well.
0.09 S.i think he is waste for rr..
0.08 Were trying to find a Chinese food place around here
0.08 Awesome, think we can get an 8th at usf some time tonight?
